# **Housing Prices Competition for Kaggle Learn Users**

# 1) Create X and y Dataframes

## Import Libraries

In [1]:
# Data Analytics Libraries
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
# Machine Learning Libraries
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

## List all files under the input directory

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


## Read the data

In [4]:
X = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv', index_col='Id')
X_test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv', index_col='Id')

## Remove rows with missing target, separate target from predictors

In [5]:
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

# 2) Clean the Data

## Remove the columns with more than half missing values

In [7]:
null_values = X.isnull().sum()
null_values[null_values>0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [8]:
# Columns with missing values more than half of the number of rows.
null_col = [col for col in X.columns if X[col].isnull().sum() > X.shape[0]/2]
null_col

['Alley', 'PoolQC', 'Fence', 'MiscFeature']

In [9]:
X.drop(null_col,axis=1,inplace=True)
X_test.drop(null_col,axis=1,inplace=True)

# 3) Feature Engineering

## Adding features 'Garage', 'Fireplace' and 'Bsmt'

Garage column: Binary Categorical
* 0 (for 81 rows having missing values in columns GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond)
* 1 (for all others rows)

In [10]:
X['Garage'] = X['GarageQual'].notnull().astype(int)
X_test['Garage'] = X_test['GarageQual'].notnull().astype(int)

Fireplace column: Binary Categorical
* 0 (for 690 rows having missing values in column FireplaceQu)
* 1 (for all others rows)

In [11]:
X['Fireplace'] = X['FireplaceQu'].notnull().astype(int)
X_test['Fireplace'] = X_test['FireplaceQu'].notnull().astype(int)

Basement column: Binary Categorical
* 0 (for 37 rows having missing values in columns BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2)
* 1 (for all others rows)

In [12]:
X['Bsmt'] = X['BsmtQual'].notnull().astype(int)
X_test['Bsmt'] = X_test['BsmtQual'].notnull().astype(int)

# 4) Feature Selection

In [13]:
# Categorical columns with low cardinality (number of unique values in a column)
categorical_cols = [cname for cname in X.columns if
                    X[cname].nunique() < 10 and 
                    X[cname].dtype == "object"]

# Numerical columns
numerical_cols = [cname for cname in X.columns if 
                X[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X = X[my_cols]
X_test = X_test[my_cols]

In [14]:
X.head(2)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,BldgType,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,Garage,Fireplace,Bsmt
Id,,,,,,,,,,,,,,,,,,,,,
1,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,0,0,0,0,0,2,2008,1,0,1
2,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Feedr,Norm,1Fam,...,0,0,0,0,0,5,2007,1,1,1


# 5) Model Creation

## Preprocessing

In [15]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [16]:
# Create object of class XGBRegressor
xgb = XGBRegressor()

# Bundle preprocessing and modeling code in a pipeline
regressor = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', xgb)
                     ])

## Visualize Pipeline

In [17]:
from sklearn import set_config
set_config(display='diagram')
regressor

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath...
                              gamma=None, gpu_id=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=None,
                              max_leaves=None, min_child_weight=None,
                              missing=nan, monotone_constraints=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, predictor=None,
                              random_state=None, reg_alpha=None,
                              reg_lambda=None, ...))])

## Grid Search & Cross Validation

In [18]:
# Hyper-Parameters that can be optimised using GridSearch
regressor.get_params().keys();

In [19]:
param_grid = [
    {        
        "model__learning_rate": [.06, .03],
        "model__n_estimators": [100, 500],
        "model__max_depth": [7, 8, 9]
    }
]
grid_search = GridSearchCV(regressor, param_grid, cv=4, verbose=1)

# 6) Training and Testing Model

In [20]:
grid_search.fit(X, y);

Fitting 4 folds for each of 12 candidates, totalling 48 fits


In [21]:
print("Best params:")
print(grid_search.best_params_)
print("Best score in grid search:")
print(grid_search.best_score_)
print("Best XGBoost on whole trained data:")
print(grid_search.score(X, y))

Best params:
{'model__learning_rate': 0.03, 'model__max_depth': 9, 'model__n_estimators': 500}
Best score in grid search:
0.8680925196913298
Best XGBoost on whole trained data:
0.9997468643062313


## Generate test predictions

In [22]:
y_preds = grid_search.predict(X_test)

## Save output to CSV file

In [23]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': y_preds.round()})
output.to_csv('submission.csv', index=False)

## Submit your results

In [24]:
output.head(2)

,Id,SalePrice
0,1461,126119.0
1,1462,151515.0
